**Contributed by:**
*Rupali K*, 
*Mahadev S*, 
*Tendool Srivatsav S*, 
*Sarang U*, 
*Roshini A*

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch


c:\Users\tendo\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


*We have a very large data without labels. So, we have to manually create the labels and it is not completed yet. So, we have used a very small part of the data to test the code but we have to create the labels manually and then finetune the model completely and then we will proceed with the deep learning models*

*Collection of data took us more time as we need more amount of medical data for this project and they are not completely open sourced, We got the access of the data we need and we are still proceeding with it*

In [ ]:

class PrescriptionDataset(Dataset):
    def __init__(self, img_folder, labels_file, processor):
        self.img_folder = img_folder
        self.processor = processor
        with open(labels_file, "r", encoding="utf-8") as f:
            self.texts = [line.strip() for line in f.readlines()]
        self.image_files = [f"{i+1}.jpg" for i in range(len(self.texts))]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_folder, self.image_files[idx])
        text = self.texts[idx]

        image = Image.open(img_path).convert("RGB")
        pixel_values = self.processor(images=image, return_tensors="pt").pixel_values.squeeze()
        labels = self.processor.tokenizer(text, return_tensors="pt").input_ids.squeeze()

        return {"pixel_values": pixel_values, "labels": labels}


model_name = "microsoft/trocr-base-handwritten"
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

if processor.tokenizer.pad_token is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token


model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.bos_token_id



img_folder = r"C:\Users\tendo\OneDrive\Documents\Srivatsav\sem 5\Big Data\Project\archive\data"
labels_file = "labels.txt"


if not os.path.exists(labels_file):
    generate_pseudo_labels(img_folder, labels_file, model_name)


train_dataset = PrescriptionDataset(img_folder, labels_file, processor)


def collate_fn(batch):
    pixel_values = torch.stack([b["pixel_values"] for b in batch])
    labels = torch.nn.utils.rnn.pad_sequence(
        [b["labels"] for b in batch],
        batch_first=True,
        padding_value=processor.tokenizer.pad_token_id
    )
    labels[labels == processor.tokenizer.pad_token_id] = -100  # ignore loss on padding
    return {"pixel_values": pixel_values, "labels": labels}



training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr-prescription-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,  
    logging_dir="./logs",
    save_total_limit=2,
    save_steps=500,
    logging_steps=50,
    predict_with_generate=True,
    fp16=torch.cuda.is_available()
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=processor.tokenizer,
    data_collator=collate_fn,
)

trainer.train()

# Save model
model.save_pretrained("./trocr-prescription-finetuned")
processor.save_pretrained("./trocr-prescription-finetuned")


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\tendo\AppData\Local\Temp\ipykernel_32356\1794566946.py:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,4.097900


[]